In [1]:
import pandas as pd
import requests, os
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from w2v_utils import *

/home/felipe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
stop_words = ["de","a","o","que","e","do","da","em","um","para",u"é","com",u"não","uma","os","no","se","na","por","mais","as","dos","como","mas","foi","ao","ele","das","tem",u"à","seu","sua","ou","ser","quando","muito",u"há","nos",u"já",u"está","eu","também",u"só","pelo","pela",u"até","isso","ela","entre","era","depois","sem","mesmo","aos","ter","seus","quem","nas","me","esse","eles",u"estão",u"você","tinha","foram","essa","num","nem","suas","meu",u"às","minha",u"têm","numa","pelos","elas","havia","seja","qual",u"será",u"nós","tenho","lhe","deles","essas","esses","pelas","este","fosse","dele","tu","te",u"vocês","vos","lhes","meus","minhas","teu","tua","teus","tuas","nosso","nossa","nossos","nossas","dela","delas","esta","estes","estas","aquele","aquela","aqueles","aquelas","isto","aquilo","estou",u"está","estamos",u"estão","estive","esteve","estivemos","estiveram","estava",u"estávamos","estavam","estivera",u"estivéramos","esteja","estejamos","estejam","estivesse",u"estivéssemos","estivessem","estiver","estivermos","estiverem","hei",u"há","havemos",u"hão","houve","houvemos","houveram","houvera","houvéramos","haja","hajamos","hajam","houvesse",u"houvéssemos","houvessem","houver","houvermos","houverem","houverei",u"houverá","houveremos",u"houverão","houveria",u"houveríamos","houveriam","sou","somos",u"são","era",u"éramos","eram","fui","foi","fomos","foram","fora",u"fôramos","seja","sejamos","sejam","fosse","fôssemos","fossem","for","formos","forem","serei",u"será","seremos",u"serão","seria",u"seríamos","seriam","tenho","tem","temos",u"tém","tinha",u"tínhamos","tinham","tive","teve","tivemos","tiveram","tivera",u"tivéramos","tenha","tenhamos","tenham","tivesse",u"tivéssemos","tivessem","tiver","tivermos","tiverem","terei",u"terá","teremos",u"terão","teria",u"teríamos","teriam", u"após", "pm","br", u"são"]

## Lendo e transformando o dataset para multilabel

In [3]:
df1 = pd.read_csv("noticias_gshow.csv", names=['id','url', 'titulo','subtitulo','corpo','entidade'])
df2 = df1.groupby(['id','url','titulo','subtitulo','corpo'])['entidade'].apply(list)
df3 = df2.to_frame().reset_index()

## Binarizando o multilabel

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer().fit(df3['entidade'])

In [5]:
Y = mlb.transform(df3['entidade'])

## Preparando o tokenizer

In [6]:
# prepare tokenizer
t = Tokenizer()
df_data = df3['titulo']
t.fit_on_texts(df_data)
vocab_size = len(t.word_index) + 1

## Encoding the docs

In [7]:
encoded_docs = t.texts_to_sequences(df_data)

In [8]:
max_size = int(df_data.str.encode(encoding='utf-8').str.len().max())

In [9]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_size, padding="post")

## Carregando o word embeddings para a memória

In [10]:
words, word_to_vec_map = read_glove_port_vecs('glove_s100.txt',100)
print("Loaded %s word vectors." % len(words))

Loaded 933716 word vectors.


## Criando a matriz de word embbeddings para o dataset

In [11]:
embedding_matrix = zeros((vocab_size, 99))
for word, i in t.word_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Definindo o modelo

In [12]:
# define model
model = Sequential()
e = Embedding(vocab_size, 99, weights=[embedding_matrix], input_length=max_size, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(Y.shape[1], activation='sigmoid'))

# compile the model 
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 133, 99)           1055736   
_________________________________________________________________
flatten_1 (Flatten)          (None, 13167)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1764)              23228352  
Total params: 24,284,088
Trainable params: 23,228,352
Non-trainable params: 1,055,736
_________________________________________________________________
None


## Treinando e avaliando o modelo

In [16]:
# fit the model
model.fit(padded_docs, Y, epochs=5, batch_size=1000)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, Y, verbose=0)

print("Accuracy: %f" % (accuracy*100))

Epoch 1/5
9953/9953 [==============================] - 1s 77us/step - loss: 0.0870 - acc: 0.9963
Epoch 2/5
9953/9953 [==============================] - 1s 77us/step - loss: 0.0807 - acc: 0.9964
Epoch 3/5
9953/9953 [==============================] - 1s 77us/step - loss: 0.0753 - acc: 0.9965
Epoch 4/5
9953/9953 [==============================] - 1s 76us/step - loss: 0.0707 - acc: 0.9966
Epoch 5/5
9953/9953 [==============================] - 1s 76us/step - loss: 0.0666 - acc: 0.9967
Accuracy: 99.668696
